In [166]:
# misc utilities and crap
import os
import zipfile
import numpy as np
from tqdm import tqdm, tqdm_notebook
import time
import image_utils

# display stuff imports
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
from IPython.display import HTML, display, clear_output

# sklearn
from sklearn.model_selection import train_test_split
import cv2

# keras stuff
import tensorflow as tf
import h5py     # needed for keras to load model weights
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

## data set

Downloading MS's gear pics:

In [18]:
! curl -O https://challenge.blob.core.windows.net/challengefiles/gear_images.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 33.4M  100 33.4M    0     0  2326k      0  0:00:14  0:00:14 --:--:-- 2488k


Now to unzip them:

In [104]:
with open('gear_images.zip', 'rb') as f:
    zf = zipfile.ZipFile(f)
    zf.extractall()

classes = os.listdir("gear_images")
NUM_CLASSES = len(classes)
print("We have:", NUM_CLASSES, " classes")
print(classes)

We have: 12  classes
['boots', 'hardshell_jackets', 'axes', 'carabiners', 'helmets', 'harnesses', 'rope', 'crampons', 'gloves', 'tents', 'insulated_jackets', 'pulleys']


So we have 12 classes, with each class image living in its own folder.
So I'm going to preprocess all these images into a processed image directory:

In [174]:
def read_all_images_in_dir(img_dir="gear_images_processed/"):
    """processes all the images in the subdirectories of the passed in img_dir
       and return a list of images and their labels."""

    images = []
    label_names = []
    labels = []
    
    for i, sub_dir in tqdm_notebook(enumerate(os.listdir(img_dir))):
        cur_dir = img_dir + sub_dir
        label = sub_dir
        label_names.append(label)
        
        for img in os.listdir(cur_dir):
            im = cv2.imread(cur_dir +"/"+ img)
            #im = cv2.resize(im, (64,64))
            images.append(im)
            labels.append(i)
    
    return np.array(images), np.array(labels), label_names

images, labels, label_names = read_all_images_in_dir()
images.shape, images[0].shape, labels[:5], label_names[:5]

((2122, 128, 128, 3),
 (128, 128, 3),
 array([0, 0, 0, 0, 0]),
 ['boots', 'hardshell_jackets', 'axes', 'carabiners', 'helmets'])

In [175]:
# Convert labels to categorical one-hot encoding
one_hot_labels = keras.utils.to_categorical(labels, num_classes=NUM_CLASSES)

X_train, X_test, y_train, y_test = train_test_split(images, one_hot_labels, test_size=0.2)
len(X_train), len(y_train), len(X_test), len(y_test)

(1697, 1697, 425, 425)

## a simple model

Preprocess images: https://keras.io/preprocessing/image/

1. Input Layer (3 channel image input layer)
2. Convolutional (2D)
3. Max Pooling
4. Convolutional (2D)
5. Max Pooling
6. Dense (Output layer)

In [176]:
model = keras.models.Sequential()

model.add(Conv2D(32, (2, 2), activation='relu', input_shape=X_train[0].shape))
model.add(Conv2D(32, (2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (2, 2), activation='relu'))
model.add(Conv2D(64, (2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(NUM_CLASSES, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 127, 127, 32)      416       
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 126, 126, 32)      4128      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 63, 63, 32)        0         
_________________________________________________________________
dropout_31 (Dropout)         (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 62, 62, 64)        8256      
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 61, 61, 64)        16448     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 30, 30, 64)        0         
__________

In [173]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test,y_test))
model.evaluate(X_test, y_test, batch_size=32)

Train on 1697 samples, validate on 425 samples
Epoch 1/10
1697/1697 [==============================] - 28s 17ms/step - loss: 15.5409 - acc: 0.0342 - val_loss: 15.1700 - val_acc: 0.0588
Epoch 2/10
1697/1697 [==============================] - 27s 16ms/step - loss: 15.6052 - acc: 0.0318 - val_loss: 15.1700 - val_acc: 0.0588
Epoch 3/10
1697/1697 [==============================] - 27s 16ms/step - loss: 15.6052 - acc: 0.0318 - val_loss: 15.1700 - val_acc: 0.0588
Epoch 4/10
1120/1697 [==================>...........] - ETA: 8s - loss: 15.6288 - acc: 0.0304

KeyboardInterrupt: 

In [ ]:
model.evaluate(X_test, y_test, batch_size=32)

## using transfer learning

Using VGG16 for now:

In [68]:
base_model = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(128, 128, 3))

for i, layer in enumerate(base_model.layers):
    layer.trainable = False
    print(i, layer.name, layer.trainable)

0 input_10 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_pool False
11 block4_conv1 False
12 block4_conv2 False
13 block4_conv3 False
14 block4_pool False
15 block5_conv1 False
16 block5_conv2 False
17 block5_conv3 False
18 block5_pool False
